In [ ]:
print SPARK_HOME

In [ ]:
words = sc.textFile("hdfs://localhost:9000/python/book.txt")

In [ ]:
words.filter(lambda w: w.startswith(" ")).take(5)

In [ ]:
counts = words.flatMap(lambda line: line.split(" ")) \
              .map(lambda word: (word, 1))           \
              .reduceByKey(lambda a, b: a + b)

In [ ]:
counts.saveAsTextFile("hdfs://localhost:9000/python/spark_output_book")

In [ ]:
counts.collect()

In [ ]:
def isprime(n):
    """
    check if integer n is a prime
    """
    # make sure n is a positive integer
    n = abs(int(n))
    # 0 and 1 are not primes
    if n < 2:
        return False
    # 2 is the only even prime number
    if n == 2:
        return True
    # all other even numbers are not primes
    if not n & 1:
        return False
    # range starts with 3 and only needs to go up the square root of n
    # for all odd numbers
    for x in range(3, int(n**0.5)+1, 2):
        if n % x == 0:
            return False
    return True

# Create an RDD of numbers from 0 to 1,000,000
nums = sc.parallelize(xrange(1000000))

# Compute the number of primes in the RDD
print nums.filter(isprime).count()

In [ ]:
import urllib2
response = urllib2.urlopen('http://www.gutenberg.org/files/30760/30760-0.txt')
html = response.read()
file("book_temp.txt","w").write(html)
words = sc.textFile("book_temp.txt")

In [ ]:
lineLengths = words.map(lambda s: len(s))
totalLength = lineLengths.reduce(lambda a, b: a + b)
totalLength

In [ ]:
from pyspark.sql import *
from pyspark.sql.types import *
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

In [ ]:
csv_dt = sc.textFile("hdfs://localhost:9000/python/BM_AIC140.csv")
header = csv_dt.take(2)
#for h in header: print h
footer = csv_dt.filter(lambda x: x not in header)
#for f in footer.collect(): print f

In [ ]:
parts = footer.map(lambda x: x.split("|"))
#for p in parts.collect(): print p
table = parts.map(lambda x: (int(x[0]), x[1].decode('utf-8')))
#for t in table.collect(): print t

In [ ]:
f_code = StructField("code", IntegerType(), True)
f_name = StructField("name", StringType(), True)
sqlCtx = SQLContext(sc)
schema = StructType([f_code, f_name])
df = sqlCtx.createDataFrame(table, schema)
df.registerTempTable("table140")

In [ ]:
results = df.select(df.code, df.name).orderBy(df.code)

In [ ]:
resultsPandas = results.toPandas()
resultsPandas

In [ ]:
fig = resultsPandas.plot(kind="bar", x="name")
#fig.set_xticklabels(df.name.decode('utf-8'))
fig

In [ ]:
plt.clf()